# Detecting Gaze Events

## What you will learn in this tutorial:

* how to detect saccades using the microsaccades algorithm
* how to detect fixations using the I-DT and I-VT algorithms

## Preparations

We import `pymovements` as the alias `pm` for convenience.

In [ ]:
import polars as pl

import pymovements as pm

Let's start by downloading our `ToyDataset` and loading in its data:

In [ ]:
dataset = pm.Dataset('ToyDataset', path='data/ToyDataset')
dataset.download()
dataset.load()

Now let's do some basic preprocessing, which will add new columns to the dataset:

* pix2deg(): adds `position` column with degrees from the screen center needed by `idt` algorithm
* dataset.pos2vel(): adds `velocity` column with X and Y gaze velocities needed by `microsaccades` and `ivt` algorithms


In [ ]:
dataset.pix2deg()
dataset.pos2vel('smooth')

dataset.gaze[0]

## Detecting Events

*pymovements* provides a range of event detection methods for several types of gaze events.

See the reference for [pymovements.events](https://pymovements.readthedocs.io/en/latest/reference/pymovements.events.html) to get an overview of all the supported methods.

For this tutorial we will use the `microsaccades` algorithm for detecting saccades and the I-DT and I-VT algorithms for detecting fixations.

We start out by detecting saccades.

In [ ]:
dataset.detect_events('microsaccades', minimum_duration=12)

The detected events are added as rows with the name `saccade` to the event dataframe:

In [ ]:
dataset.events[0]

Next we will detect fixations using the I-DT and I-VT algorithms.

To be able to differentiate between the detected events, we specify custom event names for each algorithm:

In [ ]:
dataset.detect_events('idt', dispersion_threshold=2.7, name='fixation.idt')
dataset.detect_events('ivt', velocity_threshold=20, name='fixation.ivt')

This has added new rows with the fixation events to the event dataframe.

In [ ]:
dataset.events[0].frame.filter(pl.col('name') == 'fixation.idt').head()

In [ ]:
dataset.events[0].frame.filter(pl.col('name') == 'fixation.ivt').head()

## What you have learned in this tutorial:

* detecting saccades by using the microsaccades algorithm
* detecting fixations by using the I-DT and I-VT algorithms